# IDS 2019 PROJECT

In [0]:
# Import block

import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = 'drive/My Drive/Colab Notebooks/IDSProject/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Function for reading and cleaning data

def readNClean(name): 
  my_cols = [str(i) for i in range(65)] # create some row names
  data = pd.read_csv(path+name,names=my_cols,header=None, engine="python")

  df = data.copy()

  df = df.iloc[2:]
  df.reset_index(drop=True, inplace=True)



  #dropime ebavajalikud indikaatorid 
  df.drop(df.iloc[:, 1:4], inplace = True, axis = 1)

  df = df.T.set_index(0).T

  #Transpose DateFrame - Columns to rows
  df = df.T


  #1st row to header
  df.index.names = [None]
  new_header = df.iloc[0]
  df = df[1:]
  df.columns = new_header
  
  #Remvoing Empty row 
  df.drop(df.tail(1).index,inplace=True)
  #Rounding the values
  df = df.astype(float).round(4)

  return(df)



In [0]:
#Reading and cleaning the data.

AccessToElectricity = readNClean("AccessToElectricityTotal.csv") 
AgriculturalLand = readNClean("AgriculturalLand.csv") 
ForestArea = readNClean("ForeastArea.csv")
AirPolutionMicrogramsPerCubicMeter = readNClean("AirPolutionMicrogramsPerCubicMeter.csv") 
AirTransportFreight = readNClean("AirTransportFreight.csv") 
RailwayFreight = readNClean("RailwayFreight.csv")
CO2 = readNClean("CO2Emission.csv") 
EmploymentInAgriculture = readNClean("EmpAgroculturePercOfTotalILO.csv")   # spelling mistake in file name
EmploymentInIndustry = readNClean("EmpIndustryPercOfTotalILO.csv")
EmploymentInServices = readNClean("EmpServicesPercOfTotalILO.csv") 
Unemployment = readNClean("TotalUnemploymentPerc.csv")
EnergyFromCoal = readNClean("EnergyCoal.csv") 
ElectricPowerTransprotLoss = readNClean("ElectricPowerTransprotLossPerc.csv")
EnergyConsumption = readNClean("EnergyConsuption.csv")
EnergyImports = readNClean("EnergyImportsPercOfTotalUsed.csv")
EnergyFromNaturalGas = readNClean("EnergyNaturalGas.csv")
EnergyFromOil = readNClean("EnergyOil.csv")
EnergyFromRenewable = readNClean("EnergyRenewable.csv")
EnergyFromHydro = readNClean("HydroEnergy.csv")
EnergyFromNuclear = readNClean("NuclearEnergy.csv")
ExportPercentageOfGDP = readNClean("ExportsPercOfGDP.csv")
LogisticsPerformanceIndex = readNClean("LogisticsPerformanceIndex.csv")
TotalPopulation = readNClean("TotalPopulation.csv") 
PopulationDensity = readNClean("PopDensityPerSqrkm.csv")
PopulationLivingInSlums = readNClean("PopLivingInSLums.csv")
PriceOfFuel = readNClean("PriceOfFuelUSDPerLiter.csv")
NumberOfTourists = readNClean("TourismNumberOfArrivals.csv")
UrbanPopulation = readNClean("UrbanPopulation.csv")

In [145]:
# Finding change in one value over years

def getChange(value, year1, year2):
  df = pd.DataFrame(columns = ["dif"])
  df.dif = value.loc[year2] - value.loc[year1]

  return df


# Combining all the changed values into one dataframe

def changeOverTime(year1,year2):
  changeOverTime = pd.DataFrame(columns=['Country','TotalPopulation','PopulationDensity','UrbanPopulation','PopulationLivingInSlums',
                                 'EmploymentInAgriculture','EmploymentInIndustry','EmploymentInServices','Unemployment',
                                 'NumberOfTourists','PriceOfFuel','EnergyConsumption','EnergyImports','EnergyFromCoal',
                                 'EnergyFromNaturalGas','EnergyFromOil','EnergyFromRenewable','EnergyFromHydro','EnergyFromNuclear',
                                 'ElectricPowerTransprotLoss','AccessToElectricity','AirTransportFreight','RailwayFreight','LogisticsPerformanceIndex',
                                 'ExportPercentageOfGDP','AgriculturalLand','ForestArea','AirPolutionMicrogramsPerCubicMeter','CO2'])

  changeOverTime.Country = list(TotalPopulation.columns)
  changeOverTime.TotalPopulation = list(getChange(TotalPopulation,year1,year2).dif)
  changeOverTime.PopulationDensity = list(getChange(PopulationDensity,year1,year2).dif)
  changeOverTime.UrbanPopulation = list(getChange(UrbanPopulation,year1,year2).dif)
  changeOverTime.PopulationLivingInSlums = list(getChange(PopulationLivingInSlums,year1,year2).dif)
  changeOverTime.EmploymentInAgriculture = list(getChange(EmploymentInAgriculture,year1,year2).dif)
  changeOverTime.EmploymentInIndustry = list(getChange(EmploymentInIndustry,year1,year2).dif)
  changeOverTime.EmploymentInServices = list(getChange(EmploymentInServices,year1,year2).dif)
  changeOverTime.Unemployment = list(getChange(Unemployment,year1,year2).dif)
  changeOverTime.NumberOfTourists = list(getChange(NumberOfTourists,year1,year2).dif)
  changeOverTime.PriceOfFuel = list(getChange(PriceOfFuel,year1,year2).dif)
  changeOverTime.EnergyConsumption = list(getChange(EnergyConsumption,year1,year2).dif)
  changeOverTime.EnergyImports = list(getChange(EnergyImports,year1,year2).dif)
  changeOverTime.EnergyFromCoal = list(getChange(EnergyFromCoal,year1,year2).dif)
  changeOverTime.EnergyFromNaturalGas = list(getChange(EnergyFromNaturalGas,year1,year2).dif)
  changeOverTime.EnergyFromOil = list(getChange(EnergyFromOil,year1,year2).dif)
  changeOverTime.EnergyFromRenewable = list(getChange(EnergyFromRenewable,year1,year2).dif)
  changeOverTime.EnergyFromHydro = list(getChange(EnergyFromHydro,year1,year2).dif)
  changeOverTime.EnergyFromNuclear = list(getChange(EnergyFromNuclear,year1,year2).dif)
  changeOverTime.ElectricPowerTransprotLoss = list(getChange(ElectricPowerTransprotLoss,year1,year2).dif)
  changeOverTime.AccessToElectricity = list(getChange(AccessToElectricity,year1,year2).dif)
  changeOverTime.AirTransportFreight = list(getChange(AirTransportFreight,year1,year2).dif)
  changeOverTime.RailwayFreight = list(getChange(RailwayFreight,year1,year2).dif)
  changeOverTime.LogisticsPerformanceIndex = list(getChange(LogisticsPerformanceIndex,year1,year2).dif)
  changeOverTime.ExportPercentageOfGDP = list(getChange(ExportPercentageOfGDP,year1,year2).dif)
  changeOverTime.AgriculturalLand = list(getChange(AgriculturalLand,year1,year2).dif)
  changeOverTime.ForestArea = list(getChange(ForestArea,year1,year2).dif)
  changeOverTime.AirPolutionMicrogramsPerCubicMeter = list(getChange(AirPolutionMicrogramsPerCubicMeter,year1,year2).dif)
  changeOverTime.CO2 = list(getChange(CO2,year1,year2).dif)

  return changeOverTime

changeOverTime(2004.0,2014.0)

,Country,TotalPopulation,PopulationDensity,UrbanPopulation,PopulationLivingInSlums,EmploymentInAgriculture,EmploymentInIndustry,EmploymentInServices,Unemployment,NumberOfTourists,PriceOfFuel,EnergyConsumption,EnergyImports,EnergyFromCoal,EnergyFromNaturalGas,EnergyFromOil,EnergyFromRenewable,EnergyFromHydro,EnergyFromNuclear,ElectricPowerTransprotLoss,AccessToElectricity,AirTransportFreight,RailwayFreight,LogisticsPerformanceIndex,ExportPercentageOfGDP,AgriculturalLand,ForestArea,AirPolutionMicrogramsPerCubicMeter,CO2
0,Aruba,5037.0,27.9833,-4.0,NaN,NaN,NaN,NaN,NaN,344000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,4.7713,0.0000,0.0000,NaN,-1745.492
1,Afghanistan,8644110.0,13.2404,2641373.0,NaN,-23.888,6.741,17.148,-1.352,NaN,0.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-15.1392,-0.0015,0.0000,NaN,8859.472
2,Angola,8183634.0,6.5643,6616319.0,NaN,13.611,-0.897,-12.713,-16.214,401000.0,0.37,224.6935,14.8120,0.0000,0.0000,25.2299,0.0000,-25.2299,0.000,-15.1679,4.8056,1.8250,NaN,NaN,-13.6853,1.2833,-1.0011,NaN,15969.785
3,Albania,-137835.0,-5.0304,248291.0,NaN,-2.582,-0.236,2.819,0.218,2696000.0,0.53,511.8410,-32.2058,0.0000,0.0000,-2.4973,0.0000,2.4973,0.000,12.2356,0.0000,NaN,NaN,NaN,5.9954,1.9084,-0.2671,NaN,1551.141
4,Andorra,2969.0,6.3170,865.0,NaN,NaN,NaN,NaN,NaN,-428000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,-8.5106,0.0000,NaN,-99.009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,117178.0,10.7631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.38,693.0508,-9.8799,0.3515,0.0000,-0.3610,0.0000,0.0095,0.000,-7.9556,0.0000,NaN,NaN,NaN,11.6655,NaN,NaN,NaN,NaN
260,"Yemen, Rep.",6283387.0,11.9010,3275160.0,NaN,-1.785,-1.779,3.564,1.600,743500.0,0.51,46.4128,NaN,0.0000,38.5561,-38.5561,0.0000,0.0000,0.000,3.3825,16.5100,NaN,NaN,NaN,NaN,-0.0209,0.0000,NaN,3817.347
261,South Africa,7254381.0,5.9801,7180879.0,NaN,-4.413,-5.449,9.863,-4.678,2871000.0,0.38,-391.9981,8.2089,-0.8429,0.0000,0.0758,0.7886,-0.0035,-0.018,-1.6112,5.1000,113.7493,NaN,NaN,6.0015,-0.6323,0.0000,NaN,39585.265
262,Zambia,3849111.0,5.1778,2164827.0,NaN,-16.762,3.192,13.570,-7.558,432000.0,0.58,32.1925,NaN,-0.1528,0.0000,2.4256,0.0000,-2.2728,0.000,10.6933,7.6000,83.4313,NaN,NaN,5.2824,1.2631,-2.2411,NaN,2357.881
